# tootltips

In [8]:
!curl http://localhost:9200/dpe_pre_1948_td005/_count

{"count":0,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    71  100    71    0     0   1543      0 --:--:-- --:--:-- --:--:--  1543


# Imports

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk,parallel_bulk

# Preparer la base de données pour l'analyse au moment de la sauvegarde

In [2]:
def clean_desc_txt(x):
    bad_chars_to_space = ['"', "'", "''", '""',  # these char in addr field generate errors on addok
                      ",",  # removed by precaution since we dump comma sep csv and we dont want quotechar
                      "\\n", "\\t",  #
                      "\n", "\t",  # remove carriage return/tab
                      "/", "\\", "(", ")"  # remove special separators characters and ()
                                      "[", "]", "_", "°", "^","<br>", ">", "<"
                                                                    '«', '»',

                      ]
    for bad_char in bad_chars_to_space:
        x=x.replace(bad_char,' ')
    return x

In [3]:
def setup_es_client():
    es_client = Elasticsearch()

    index_name = "dpe_extract_text"
    index_configurations = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "max_result_window": 5000000,

            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "text_to_analyze": {
                    "type": "text",
                    "analyzer": "french"      
                },
                "category_index":{
                    'type':'keyword'
                }
            }
        }
    }

    try:
        es_client.indices.delete(
            index=index_name,
        )
    except:
        pass
    es_client.indices.create(
        index=index_name,
        body=index_configurations
    )
    return es_client

# fonctions

In [4]:
from generate_dpe_annexes_scripts.utils import strip_accents

def gendata(index_name,id_data,data):
    for id_,vr in zip(id_data,data):
        yield {
            "_index": index_name,
    "id": id_,
    "text_to_analyze": vr
        }

def search_from_search_dict(es_client,search_dict,index_name):
    
    s_list = list()
    for label,char_list in  search_dict.items():
        new_char_list=list()
        for char in char_list:
            if isinstance(char,str):
                new_char_list.append(char)
            if isinstance(char,tuple):
                new_char_list.append('('+') OR ('.join(char)+')')
        search_instruction = '('+') AND ('.join(new_char_list) +')'
        search_body = {
            "query": {
                "query_string": {
                    "query": search_instruction,
                    "default_field": "text_to_analyze"
                },

            },

        }
        #es_client.count(index=index_name, body=search_body)

        a_dict=es_client.search(index=index_name, body=search_body,size=500000)

        hits = a_dict['hits']['hits']

        s=pd.Series(index=[el['_source']['id'] for el in hits])
        s[:]=label
        s_list.append(s)
    s_all=pd.concat(s_list)

    return s_all

def search_and_affect(data,id_col,val_col,search_dict,index_name='dpe_extract_text',agg_method='keep_first'):
    es_client = setup_es_client()
    bulk(es_client, gendata(index_name,data[id_col],data[val_col]))
    s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

    df=s_all.to_frame('label')

    df.index.name = 'id'
    df=df.reset_index()

    df['category']=df.label.replace(reverse_cat_gen_ch)

    df['label']=pd.Categorical(df.label,categories=list(gen_ch_search_dict_flat.keys())+['indetermine'],ordered=True)

    df_drop = df.sort_values('label').drop_duplicates(subset=['id','category'])
    
    df_drop.label = df_drop.label.fillna('indetermine')
# #     grp = df_drop.groupby('id').label.apply(lambda x: ' + '.join(sorted(list(set(x))))).reset_index()
# #     m=data.merge(grp,on='id',how='left')

#     m.label=m.label.fillna('indetermine')
    return df_drop

def agg_concat_label_text(table,col_label='label',col_vr='valeur_renseignee'):
    table[[col_label,col_vr]]=table[[col_label,col_vr]].fillna('indetermine')
    agg = table.groupby('td001_dpe_id').agg({

    col_label:lambda x: ' + '.join(sorted(list(set(x)))),
    col_vr:lambda x: ' + '.join(sorted(list(set(x))))
    })

    for col in agg:
        agg[col] =  agg[col].apply(lambda x:' + '.join(sorted(list(set([el for el in x.split(' + ') if el!='indetermine'])))))
        agg[col] = agg[col].replace('','indetermine')
    return agg

# load tables

In [5]:
import numpy as np

import pandas as pd
from pathlib import Path
import json
from generate_dpe_annexes_scripts.utils import round_float_cols, unique_ordered
from config import paths


if __name__ == '__main__':

    data_dir = paths['DPE_DEPT_PATH']
    annexe_dir = paths['DPE_DEPT_ANNEXE_PATH']
    annexe_dir = Path(annexe_dir)
    annexe_dir.mkdir(exist_ok=True, parents=True)
    for dept_dir in Path(data_dir).iterdir():
        if dept_dir.name=='94':
            print(dept_dir)
            annexe_dept_dir = annexe_dir / dept_dir.name
            annexe_dept_dir.mkdir(exist_ok=True, parents=True)
            # LOAD TABLES
            td001 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td003 = pd.read_csv(dept_dir / 'td003_descriptif.csv', dtype=str)
            td005 = pd.read_csv(dept_dir / 'td005_fiche_technique.csv', dtype=str)
            td006 = pd.read_csv(dept_dir / 'td001_dpe.csv', dtype=str)
            td012_p = pd.read_csv(annexe_dept_dir / 'td012_generateur_chauffage_annexe.csv', dtype=str)
            td014_p = pd.read_csv(annexe_dept_dir / 'td014_generateur_ecs_annexe.csv', dtype=str)
            td001_sys_ch= pd.read_csv(annexe_dept_dir / 'td001_sys_ch_agg_annexe.csv', dtype=str)
            td001_sys_ecs= pd.read_csv(annexe_dept_dir / 'td001_sys_ecs_agg_annexe.csv', dtype=str)

            gen= pd.read_csv(annexe_dept_dir / 'td001_annexe_generale.csv', dtype=str)
            td001_sys_ch.columns = [col.replace('_chauffage','_ch') for col in td001_sys_ch]
            
            print(dept_dir)
            break
            # ENVELOPPE PROCESSING


D:\data\dpe_full\depts\94
D:\data\dpe_full\depts\94


In [6]:
td001_sys_ch.type_installation_ch_concat=td001_sys_ch.type_installation_ch_concat.str.replace('Chauffage Individuel',
                                                                                              'individuel').str.replace('Chauffage Collectif',
                                                                                                                        'collectif')

# dicos


In [7]:
# elements de recherche

mixte = ('mixte', 'combine', 'chauffage ecs','lie ','combine ',"idem")
elec =('electricite', 'electrique','electriques','elec.','joule','joules')
reseau_chaleur = ('"reseau de chaleur"','"reseaux de chaleurs"','"reseaux de chaleur"','"reseau chaleur"','"chauffage urbain"')
pac = ('pompe AND chaleur', 'pac ', 'thermodynamique')
type_chaudiere = dict(zip(['standard', 'basse temperature', 'condensation'],
                                               [('standard', 'classique'), '"basse temperature"',
                                                ('condensation', 'condenseurs')]))

chauffe_bain=("individuelle AND ballon", "chauffe-eau", "accumulateur", "chauffe AND bain",'chauffe-bain')

In [8]:
# dictionnaires annexes

installation_dict={'Chauffage Collectif':[('collective','collectif')],
                                 'Chauffage Individuel':[('individuelle','individuel')],
                                }

energie_dict = {
               'Gaz naturel':['gaz'],
               'Electricité non renouvelable':[elec],
                'Réseau de chaleurs':reseau_chaleur,
                'Fioul domestique':['fioul'],
    'Bois, biomasse':[('bois','biomasse')],
    'Gaz propane ou butane':[('propane','butane','gpl')],
    'Charbon':['charbon'],               
               }


energie_dict_lower = {
               'Gaz naturel':'gaz',
               'Electricité non renouvelable':'fioul',
                'Réseau de chaleurs':'reseau de chaleur',
                'Fioul domestique':'fioul',
    'Bois, biomasse':'bois',
    'Gaz propane ou butane':'autre(gpl/butane/propane)',
    'Charbon':'charbon',               
               }

energie_mods = energie_dict.keys()
energie_chaudiere_mods = ['gaz','fioul','autre(gpl/butane/propane)']
type_chaudiere_mods = ['condensation','basse temperature','standard']

bois = ('bois', 'biomasse','flamme verte')

In [23]:


# DICTIONNAIRE DE MATCHING TEXTE SYSTEME : ECS -> L'ORDRE A DE L'IMPORTANCE : 
# la première affectation est considérée comme la bonne


gen_ecs_search_dict = {
    "ecs thermodynamique electrique(PAC ou ballon)": [
        ('pompe AND chaleur', 'pac', 'thermodynamique', '"air extrait"', '"air exterieur"', '"air ambiant"')],
    "ballon a accumulation electrique": [('ballon', 'classique', 'accumulation'), elec],


    

}

for type_chaudiere, type_chaudiere_keys in zip(type_chaudiere_mods+ ['non déterminee'],
                                               [('standard', 'classique'), '"basse temperature"',
                                                ('condensation', 'condenseurs'), None]):
    for energie in energie_chaudiere_mods:
        energie_keywords = energie
        if energie == 'autre(gpl/butane/propane)':
            energie_keywords = ('gpl', 'butane', 'propane')
        if type_chaudiere_keys is not None:
            gen_ecs_search_dict[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords,
                                                                                            type_chaudiere_keys]
        else:
            gen_ecs_search_dict[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords
                                                                                            ]
    gen_ecs_search_dict[f'chaudiere energie indetermine {type_chaudiere}']=['chaudiere',type_chaudiere_keys] 


gen_ecs_search_dict.update({  
        "ecs collective reseau chaleur": [reseau_chaleur],
    'production mixte gaz': ["gaz", mixte],
    'production mixte fioul': ["fioul", mixte],
    'production mixte indetermine': [ mixte],
 'chauffe-eau gaz independant': [chauffe_bain, "gaz"],
    'chauffe-eau gpl independant': [chauffe_bain, "gpl"],
    'chauffe-eau fioul independant': [chauffe_bain,
                                      "fioul"],
    'chaudiere electrique': ["chaudiere",elec],
    "ecs instantanee electrique": ['instantanee', elec],    
    "ecs electrique indetermine": [elec],
    'chaudiere  indetermine': ["chaudiere"],
    "chauffe-eau independant indetermine":[("individuelle AND ballon", "chauffe-eau", "accumulateur", "chauffe AND bain")],
        

})

In [10]:
# DICTIONNAIRE DE MATCHING TEXTE : CHAUFFAGE -> L'ORDRE A DE L'IMPORTANCE : 
# la première affectation est considérée comme la bonne

gen_ch_search_dict=dict()

gen_ch_search_dict['pac'] = { "pac geothermique en releve de chaudiere": [pac, (
                                           'geothermique', 'geothermie'),'chaudiere'],
                                         "pac eau/eau en releve de chaudiere": [pac, 'eau','chaudiere'],
                                          "pac air/eau en releve de chaudiere": [pac, 'air AND eau','chaudiere'],
                                            "pac geothermique": [pac, (
                                           'geothermique', 'geothermie')],
                                         "pac air/eau": [pac, 'air AND eau'],
                                         "pac eau/eau": [pac, 'eau'],
                                        "pac air/air": [pac, ('air','split')],}

gen_ch_search_dict['poele'] = {"poele ou insert bois": [('poele', 'insert'), bois],
                                       "poele ou insert fioul/gpl": [('poele', 'insert'), ('fioul', 'gpl')],}




chaudiere_dict_ch = dict()

chaudiere_dict_ch['chaudiere bois']=[('"chaudiere bois"'
                                               ,'"chaudiere a bois"'
                                               ,'"chaudiere bois/biomasse"'
                                               ,'"chaudiere a bois/biomasse"')]
chaudiere_dict_ch['chaudiere electrique'] = ['"chaudiere electrique"']

for type_chaudiere, type_chaudiere_keys in zip(type_chaudiere_mods+ ['indetermine'],
                                               [('condensation', 'condenseurs'), '"basse temperature"',
                                                ('standard', 'classique'), None]):
    

    for energie in energie_chaudiere_mods:
        energie_keywords = energie
        if energie == 'autre(gpl/butane/propane)':
            energie_keywords = ('gpl', 'butane', 'propane')
        if type_chaudiere_keys is not None:
            chaudiere_dict_ch[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie_keywords,
                                                                                            type_chaudiere_keys]
        else:
            chaudiere_dict_ch[f'chaudiere {energie} {type_chaudiere}'] = [('chaudiere','collectif'), energie_keywords
                                                                                            ]
    chaudiere_dict_ch[f'chaudiere energie indetermine {type_chaudiere}']=['chaudiere',type_chaudiere_keys] 
    



# les radiateurs gaz sont inclus avec les chaudieres car ces systèmes sont a priori exclusifs les un des autres. 
chaudiere_dict_ch['radiateurs gaz']=[('radiateur','radiateurs'), 'gaz'] 


gen_ch_search_dict['chaudiere']=chaudiere_dict_ch
gen_ch_search_dict['reseau_chaleur']={"reseau de chaleur": [reseau_chaleur],}


gen_ch_search_dict['effet_joule']={'radiateurs electriques': [('radiateur','radiateurs'), elec],
    "convecteurs bi-jonction": [('bi AND jonction','bijonction','bi-jonction')],                           
'panneaux rayonnants electriques nfc': [('panneau','panneaux'),
                                       ('rayonnant','rayonnants'),
                                       ('nf','nfc')],
'plafonds/planchers rayonnants electriques nfc': [('plancher', 'plafond',
                                                  'planchers','plafonds'),
                                                 ('rayonnant','rayonnants')],
"convecteurs electriques nfc": [('convecteur','convecteurs'),
                                                                       ('nf','nfc')],
                                   
    "autres emetteurs a effet joule": [elec],}


gen_ch_search_dict_flat = dict()
{gen_ch_search_dict_flat.update(v) for k,v in gen_ch_search_dict.items() }

reverse_cat_gen_ch = dict()

for cat,v in gen_ch_search_dict.items():
    for label in v:
        reverse_cat_gen_ch[label]=cat
        


In [11]:
from generate_dpe_annexes_scripts.td011_td012_processing import gen_ch_normalized_lib_matching_dict

In [27]:
from generate_dpe_annexes_scripts.td013_td014_processing import gen_ecs_normalized_lib_matching_dict

In [12]:
gen_ch_search_dict_flat.keys() -set(gen_ch_normalized_lib_matching_dict.keys())

{'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine indetermine',
 'chaudiere energie indetermine standard',
 'pac air/eau en releve de chaudiere',
 'pac eau/eau en releve de chaudiere',
 'pac geothermique en releve de chaudiere',
 'radiateurs gaz'}

In [29]:
gen_ecs_search_dict.keys() -set(gen_ecs_normalized_lib_matching_dict.keys())

{'chaudiere  indetermine',
 'chaudiere autre(gpl/butane/propane) non déterminee',
 'chaudiere electrique',
 'chaudiere energie indetermine basse temperature',
 'chaudiere energie indetermine condensation',
 'chaudiere energie indetermine non déterminee',
 'chaudiere energie indetermine standard',
 'chaudiere fioul non déterminee',
 'chaudiere gaz non déterminee',
 'chauffe-eau independant indetermine',
 'ecs collective reseau chaleur',
 'production mixte fioul',
 'production mixte gaz',
 'production mixte indetermine'}

In [13]:
set(gen_ecs_normalized_lib_matching_dict.keys())

{'autres emetteurs a effet joule',
 'chaudiere autre(gpl/butane/propane) basse temperature',
 'chaudiere autre(gpl/butane/propane) condensation',
 'chaudiere autre(gpl/butane/propane) indetermine',
 'chaudiere autre(gpl/butane/propane) standard',
 'chaudiere bois',
 'chaudiere electrique',
 'chaudiere fioul basse temperature',
 'chaudiere fioul condensation',
 'chaudiere fioul indetermine',
 'chaudiere fioul standard',
 'chaudiere gaz basse temperature',
 'chaudiere gaz condensation',
 'chaudiere gaz indetermine',
 'chaudiere gaz standard',
 'convecteurs bi-jonction',
 'convecteurs electriques nfc',
 'pac air/air',
 'pac air/eau',
 'pac eau/eau',
 'pac geothermique',
 'panneaux rayonnants electriques nfc',
 'plafonds/planchers rayonnants electriques nfc',
 'poele ou insert bois',
 'poele ou insert fioul/gpl',
 'radiateurs electriques',
 'reseau de chaleur'}

In [14]:
set(gen_ch_normalized_lib_matching_dict.keys())-set(gen_ch_search_dict_flat.keys())

set()

# ECS

In [21]:

td005_ecs = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '17']
td005_ecs.descriptif = td005_ecs.valeur_renseignee.fillna('indetermine')

td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ecs.valeur_renseignee = td005_ecs.valeur_renseignee.apply(lambda x:clean_desc_txt(x))



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


In [24]:

m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ecs_search_dict)

gen_ecs_lib_ft=m.merge(td005_ecs[['id','td001_dpe_id']])


m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']])



m=search_and_affect(td005_ecs,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict,agg_method='concat')

energie_ecs_ft=m.merge(td005_ecs[['id','td001_dpe_id']])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [25]:
td003_ecs = td003.loc[td003.tr007_type_descriptif_id == '10']
td003_ecs.descriptif = td003_ecs.descriptif.fillna('indetermine')
td003_ecs.descriptif = td003_ecs.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ecs.descriptif = td003_ecs.descriptif.apply(lambda x:clean_desc_txt(x))



In [26]:

m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=gen_ecs_search_dict)

gen_ecs_lib_desc=m.merge(td003_ecs[['id','td001_dpe_id']])


m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']])



m=search_and_affect(td003_ecs,id_col='id',val_col='descriptif',
                    search_dict=energie_dict,agg_method='concat')

energie_ecs_desc=m.merge(td003_ecs[['id','td001_dpe_id']])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


# Chauffage

In [15]:

td005_ch = td005.loc[td005.tr011_sous_categorie_fiche_technique_id == '16']
td005_ch.descriptif = td005_ch.valeur_renseignee.fillna('indetermine')

td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.str.lower().apply(lambda x: strip_accents(x))
td005_ch.valeur_renseignee = td005_ch.valeur_renseignee.apply(lambda x:clean_desc_txt(x))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [17]:

m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=gen_ch_search_dict_flat)

gen_ch_lib_ft=m.merge(td005_ch[['id','td001_dpe_id']])


m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=installation_dict)

type_installation_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']])



m=search_and_affect(td005_ch,id_col='id',val_col='valeur_renseignee',
                    search_dict=energie_dict,agg_method='concat')

energie_ch_ft=m.merge(td005_ch[['id','td001_dpe_id']])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [18]:
td003_ch = td003.loc[td003.tr007_type_descriptif_id == '11']
td003_ch.descriptif = td003_ch.descriptif.fillna('indetermine')
td003_ch.descriptif = td003_ch.descriptif.str.lower().apply(lambda x: strip_accents(x))
td003_ch.descriptif = td003_ch.descriptif.apply(lambda x:clean_desc_txt(x))
  

In [19]:

m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=gen_ch_search_dict_flat)

gen_ch_lib_desc=m.merge(td003_ch[['id','td001_dpe_id']])


m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=installation_dict)

type_installation_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']])



m=search_and_affect(td003_ch,id_col='id',val_col='descriptif',
                    search_dict=energie_dict,agg_method='concat')

energie_ch_desc=m.merge(td003_ch[['id','td001_dpe_id']])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


# fusion

## méthodologie de fusion

* on priorise les informations déclarées dans le modèle de données 3CL

* on complète les données manquantes avec les données FT puis descriptif

* toutes les données indeterminées sont complétées. 


## generateurs chauffage


### préparation de la donnée

In [424]:
gen_ch_lib_from_txt=pd.concat([gen_ch_lib_desc[['label','category','td001_dpe_id']],
                               gen_ch_lib_ft[['label','category','td001_dpe_id']]],axis=0)

gen_ch_lib_from_txt =gen_ch_lib_from_txt.sort_values(by=['td001_dpe_id','category','label'])

gen_ch_lib_from_txt = gen_ch_lib_from_txt.drop_duplicates(subset=['category','td001_dpe_id'],keep='first')

gen_ch_lib_from_txt['source']='txt'

In [425]:
gen_ch_lib_from_data = td012_p[['td001_dpe_id','gen_ch_lib_infer']].copy()

In [426]:
gen_ch_lib_from_data.columns=['td001_dpe_id','label']

In [427]:
gen_ch_lib_from_data['category']=gen_ch_lib_from_data.label.replace(reverse_cat_gen_ch)

gen_ch_lib_from_data['label']=pd.Categorical(gen_ch_lib_from_data.label,categories=list(gen_ch_search_dict_flat.keys())+['indetermine'],ordered=True)
gen_ch_lib_from_data['source']='data'

In [428]:
gen_ch_lib = pd.concat([gen_ch_lib_from_data,gen_ch_lib_from_txt],axis=0)

gen_ch_lib['source']=pd.Categorical(gen_ch_lib.source,categories=['data','txt'],ordered=True)
gen_ch_lib['category']=pd.Categorical(gen_ch_lib.category,categories=list(gen_ch_search_dict.keys()),ordered=True)



### amélioration des labels chaudières

In [429]:
is_chaudiere_ener_ind = gen_ch_lib.label.str.contains('chaudiere energie indetermine')
is_chaudiere_ener_ind = is_chaudiere_ener_ind &(~gen_ch_lib.label.str.contains('chaudiere energie indetermine indetermine'))
is_chaudiere_ind = gen_ch_lib.label.str.contains('chaudiere')&gen_ch_lib.label.str.contains('indetermine')&(~is_chaudiere_ener_ind)


In [430]:
gen_ch_lib['chaudiere_ener_ind']=is_chaudiere_ener_ind
gen_ch_lib['chaudiere_ind']=is_chaudiere_ind

In [431]:
sum_inds=gen_ch_lib.groupby('td001_dpe_id')[['chaudiere_ener_ind','chaudiere_ind']].sum()

In [432]:
id_sel=sum_inds.loc[(sum_inds>0).min(axis=1)].index

In [442]:
ind_sel = gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ind
chaudiere_ind=gen_ch_lib.loc[ind_sel]

chaudiere_ener_ind=gen_ch_lib.loc[gen_ch_lib.td001_dpe_id.isin(id_sel)&gen_ch_lib.chaudiere_ener_ind]

In [443]:
suffix=chaudiere_ener_ind.set_index('td001_dpe_id').label.apply(lambda x:x.split('energie indetermine')[-1].strip())
suffix.name='suffix'

In [444]:
chaudiere_ind = chaudiere_ind.merge(suffix,on='td001_dpe_id',how='left')

In [445]:
chaudiere_ind.label =chaudiere_ind.label.astype(str)+ ' '+chaudiere_ind.suffix

In [446]:
chaudiere_ind.label=chaudiere_ind.label.str.replace('indetermine ','').str.strip()

In [447]:
gen_ch_lib.loc[ind_sel,'label']=chaudiere_ind.label.values

In [448]:
gen_ch_lib.loc[ind_sel,'label']

57       chaudiere gaz basse temperature
75            chaudiere gaz condensation
92            chaudiere gaz condensation
114      chaudiere gaz basse temperature
115           chaudiere gaz condensation
                      ...               
49721         chaudiere gaz condensation
49726         chaudiere gaz condensation
49733         chaudiere gaz condensation
49735         chaudiere gaz condensation
49739         chaudiere gaz condensation
Name: label, Length: 396, dtype: category
Categories (36, object): [pac geothermique en releve de chaudiere < pac eau/eau en releve de chaudiere < pac air/eau en releve de chaudiere < pac geothermique ... plafonds/planchers rayonnants electriques nfc < convecteurs electriques nfc < autres emetteurs a effet joule < indetermine]

In [359]:
multiple_ind = sel.groupby(['td001_dpe_id']).label.count()


In [362]:
multiple_ind_id=multiple_ind[multiple_ind>1].index

In [365]:
sel.loc[sel.td001_dpe_id.isin(multiple_ind_id)].sort_values(by='td001_dpe_id')

,td001_dpe_id,label,category,source,chaudiere_ind,chaudiere_not_ind
5353,1001047,chaudiere gaz indetermine,chaudiere,data,True,False
19890,1001047,chaudiere energie indetermine condensation,chaudiere,txt,True,False
211117,1002357,chaudiere energie indetermine indetermine,chaudiere,txt,True,False
5360,1002357,chaudiere gaz indetermine,chaudiere,data,True,False
5383,1011045,chaudiere gaz indetermine,chaudiere,data,True,False
...,...,...,...,...,...,...
5334,997115,chaudiere fioul indetermine,chaudiere,data,True,False
209335,997168,chaudiere energie indetermine indetermine,chaudiere,txt,True,False
5336,997168,chaudiere gaz indetermine,chaudiere,data,True,False
5342,998777,chaudiere gaz indetermine,chaudiere,data,True,False


### fusion

In [ ]:
gen_ch_lib = gen_ch_lib.sort_values(by=['td001_dpe_id','category','label','source'])

gen_ch_lib = gen_ch_lib.drop_duplicates(subset=['td001_dpe_id','category'])

gen_ch_lib=gen_ch_lib.groupby('td001_dpe_id').label.apply(lambda x:' + '.join(np.unique(x)))

In [316]:
td001_sys=td001.rename(columns ={'id':'td001_dpe_id'})[['td001_dpe_id']].merge(td001_sys_ch[['td001_dpe_id','gen_ch_lib_infer_concat']],on='td001_dpe_id',
                                                                              how='left')


In [317]:
td001_sys = td001_sys.merge(gen_ch_lib.reset_index(),on='td001_dpe_id',how='left')

In [318]:
td001_sys

,td001_dpe_id,gen_ch_lib_infer_concat,label
0,212,reseau de chaleur,reseau de chaleur
1,304,chaudiere gaz indetermine,chaudiere gaz indetermine
2,566,convecteurs electriques nfc,convecteurs electriques nfc
3,898,reseau de chaleur,reseau de chaleur
4,1500,convecteurs electriques nfc,panneaux rayonnants electriques nfc
...,...,...,...
145974,9429928,NaN,chaudiere gaz indetermine
145975,9430782,chaudiere gaz indetermine,chaudiere gaz indetermine
145976,9430797,NaN,chaudiere gaz indetermine
145977,9431074,NaN,chaudiere gaz indetermine


In [319]:
diff=td001_sys.gen_ch_lib_infer_concat!=td001_sys.label

### affectation des libéllés manquants via fiches techniques et descriptifs pour les generateurs

In [118]:
gen_ch_libs = ['gen_ch_lib_concat_final','gen_ch_lib_infer_concat','gen_ch_lib_ft','gen_ch_lib_desc','vr_source_ft','vr_source_desc']

In [119]:
m['gen_ch_lib_concat_final']=m['gen_ch_lib_infer_concat'].fillna('indetermine')

is_ind = m.gen_ch_lib_concat_final=='indetermine'

In [120]:
is_ind.mean()

0.6833517149727015

#### cas ou on a une source de vérité unique

In [121]:
desc_ind=m.gen_ch_lib_desc=='indetermine'
ft_ind = m.gen_ch_lib_ft=='indetermine'

sub_ft = is_ind&desc_ind&(~ft_ind)
sub_desc = is_ind&ft_ind&(~desc_ind)

m.loc[sub_ft,'gen_ch_lib_concat_final']=m.loc[sub_ft,'gen_ch_lib_ft']
m.loc[sub_desc,'gen_ch_lib_concat_final']=m.loc[sub_desc,'gen_ch_lib_desc']

#### cas ou on a de l'information sur ft et desc mais il faut choisir

In [122]:

desc_cont_ind = m.gen_ch_lib_desc.str.contains('indetermine')&(is_ind)
ft_cont_ind = m.gen_ch_lib_ft.str.contains('indetermine')&(is_ind)

# on prend le desc si pas de indetermine 
sel_desc=ft_cont_ind&(~desc_cont_ind)
m.loc[sel_desc,'gen_ch_lib_concat_final']=m.loc[sel_desc,'gen_ch_lib_desc']
sel_ft=(~ft_cont_ind)&desc_cont_ind
m.loc[sel_ft,'gen_ch_lib_concat_final']=m.loc[sel_ft,'gen_ch_lib_ft']

# si meme label on prend l'un ou l'autre et c'est plus fiable
same_label =  (m.gen_ch_lib_ft==m.gen_ch_lib_desc)&(is_ind)
m.loc[same_label,'gen_ch_lib_concat_final']=m.loc[same_label,'gen_ch_lib_ft']


In [123]:
is_ind = m.gen_ch_lib_concat_final=='indetermine'

In [137]:
m[is_ind].gen_ch_lib_ft.value_counts()

indetermine                                                                 5790
radiateurs electriques + reseau de chaleur                                  1009
convecteurs electriques nfc                                                  820
radiateurs electriques                                                       720
panneaux rayonnants electriques nfc                                          509
                                                                            ... 
chaudiere bois/biomasse + poele ou insert bois                                 1
plafonds/planchers rayonnants electriques nfc + poele ou insert bois           1
pac air/eau + poele ou insert bois + radiateurs electriques                    1
chaudiere energie indetermine basse temperature + radiateurs electriques       1
chaudiere fioul indetermine + radiateurs electriques                           1
Name: gen_ch_lib_ft, Length: 83, dtype: int64

In [75]:
m.loc[sub_desc,gen_ch_libs]

,gen_ch_lib_concat_final,gen_ch_lib_infer_concat,gen_ch_lib_ft,gen_ch_lib_desc
240,reseau de chaleur,NaN,indetermine,reseau de chaleur
573,reseau de chaleur,NaN,indetermine,reseau de chaleur
865,reseau de chaleur,NaN,indetermine,reseau de chaleur
1024,reseau de chaleur,NaN,indetermine,reseau de chaleur
1045,NaN,NaN,indetermine,NaN
...,...,...,...,...
121693,NaN,NaN,indetermine,NaN
121722,NaN,NaN,indetermine,NaN
122619,NaN,NaN,indetermine,NaN
122755,NaN,NaN,indetermine,NaN


### traitement des chaudières

#### redressement des libellés chaudières avec fiches techniques et descriptifs pour le chauffage

In [51]:
is_chaudiere = m.gen_ch_lib_infer_concat.str.contains('chaudiere')

is_not_type = ~m.gen_ch_lib_infer_concat.fillna('indetermine').str.contains('|'.join(type_chaudiere_mods)) 
is_not_type = is_not_type & m.gen_ch_lib_infer_concat.str.contains('indetermine')

In [57]:
is_chaudiere_ft = m.gen_ch_lib_ft.str.contains('chaudiere')

is_type_ft = m.gen_ch_lib_ft.fillna('indetermine').str.contains('|'.join(type_chaudiere_mods)) 

is_ener_ft = m.gen_ch_lib_ft.fillna('indetermine').str.contains('|'.join(type_chaudiere_mods)) 


In [ ]:
sub = is_chaudiere & is_not_type&is_chaudiere_ft&is_type_ft

In [53]:
m.loc[,'gen_ch_lib_concat_corr']

15                                chaudiere gaz indetermine
30                                chaudiere gaz indetermine
93                                chaudiere gaz indetermine
127                               chaudiere gaz indetermine
130                               chaudiere gaz indetermine
                                ...                        
124339    autres emetteurs a effet joule + chaudiere fio...
124340                            chaudiere gaz indetermine
124349                            chaudiere gaz indetermine
124351                            chaudiere gaz indetermine
136121                            chaudiere gaz indetermine
Name: gen_ch_lib_concat_corr, Length: 15306, dtype: object

# debug

In [298]:
td005_ch.loc[(td005_ch.td001_dpe_id=='2253004')].valeur_renseignee.to_dict()

{545454: 'chaudiere individuelle gaz naturel basse temperature installee apres 2000 avec thermostat d ambiance  reseau isole  systeme basse temperature<br  re : 1  rr : 1  rd : 1  rg : 0  pn : 24  fch : 0',
 545461: 'pompe a chaleur air eau avec thermostat d ambiance  systeme individuel)<br  re : 1  rr : 1  rd : 0 9  rg : 2 6  pn : 0  fch : 0'}

In [355]:
m.gen_ch_lib_ft.value_counts()

autres emetteurs a effet joule                                                                                24626
chaudiere gaz indetermine                                                                                     18993
reseau de chaleur                                                                                             18343
chaudiere gaz basse temperature                                                                               12307
indetermine                                                                                                   11710
                                                                                                              ...  
panneaux rayonnants electriques nfc + plafonds/planchers rayonnants electriques nfc + poele ou insert bois        1
autres emetteurs a effet joule + chaudiere bois + pac air/air                                                     1
chaudiere fioul basse temperature + pac air/eau                         

In [354]:
m.loc[m.gen_ch_lib_ft=='pac air/eau + reseau de chaleur'].to_dict(orient='records')

[]

In [175]:
toto="""
chaudiere individuelle gaz naturel basse temperature installee apres 2000 avec thermostat d ambiance  reseau isole  systeme basse temperature<br  re : 1  rr : 1  rd : 1  rg : 0  pn : 24  fch : 0
pompe a chaleur air eau avec thermostat d ambiance  systeme individuel)<br  re : 1  rr : 1  rd : 0 9  rg : 2 6  pn : 0  fch : 0
"""

In [176]:
index_name = 'dpe_extract_text'

In [197]:
search_instruction = '((reseau de chaleur) OR (reseaux de chaleurs) OR (reseaux de chaleur))'

search_instruction = '("chaudiere individuelle gaz") AND NOT (thermostat AND ambianceqsdqs)'

In [198]:
#search_instruction = 'energie'

In [199]:
import time

In [200]:
search_body = {
    "query": {
        "query_string": {
            "query": search_instruction,
            "default_field": "text_to_analyze"
        },

    },

}

es_client = setup_es_client()
bulk(es_client, gendata(index_name,['id'],[toto]))

time.sleep(1)
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8697597,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': '2dcOcXYBwqyqBEdH5ca8',
    '_score': 0.8697597,
    '_source': {'id': 'id',
     'text_to_analyze': '\nchaudiere individuelle gaz naturel basse temperature installee apres 2000 avec thermostat d ambiance  reseau isole  systeme basse temperature<br  re : 1  rr : 1  rd : 1  rg : 0  pn : 24  fch : 0\npompe a chaleur air eau avec thermostat d ambiance  systeme individuel)<br  re : 1  rr : 1  rd : 0 9  rg : 2 6  pn : 0  fch : 0\n'}}]}}

In [201]:
a_dict=es_client.search(index=index_name, body=search_body,size=500000)
a_dict

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.8697597,
  'hits': [{'_index': 'dpe_extract_text',
    '_type': '_doc',
    '_id': '2dcOcXYBwqyqBEdH5ca8',
    '_score': 0.8697597,
    '_source': {'id': 'id',
     'text_to_analyze': '\nchaudiere individuelle gaz naturel basse temperature installee apres 2000 avec thermostat d ambiance  reseau isole  systeme basse temperature<br  re : 1  rr : 1  rd : 1  rg : 0  pn : 24  fch : 0\npompe a chaleur air eau avec thermostat d ambiance  systeme individuel)<br  re : 1  rr : 1  rd : 0 9  rg : 2 6  pn : 0  fch : 0\n'}}]}}

In [133]:
bulk(es_client, gendata(index_name,['id'],[toto]))


(1, [])

In [111]:
es_client

<Elasticsearch([{}])>

In [110]:
a_dict

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [ ]:

s_all = search_from_search_dict(es_client,search_dict,index_name=index_name)

In [46]:
m_ch = m

In [81]:
id_=m_ch.vr_source_ft.str.contains('air')&m_ch.vr_source_ft.str.contains('eau')&m_ch.vr_source_ft.str.contains('pac')

In [82]:
m_ch.loc[id_].to_dict(orient='records')

[{'td001_dpe_id': '1009801',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split multisplit  energie : electrique)  avec equipement d intermittence central sans minimum de temperature  absence de regulation par piece  reseau de distribution isole',
  'type_installation_ch_ft': 'indetermine',
  'energie_ch_ft': 'Electricité non renouvelable',
  'gen_ch_lib_desc': 'pac air/air',
  'vr_source_desc': 'installation de chauffage \r - chauffage 1  pac air air split multisplit  energie electrique  reseau individuel',
  'type_installation_ch_desc': 'Chauffage Individuel',
  'energie_ch_desc': 'Chauffage Individuel',
  'gen_ch_lib_infer_concat': nan,
  'mix_energetique_ch': nan,
  'type_installation_ch_concat': nan,
  'nom_methode_dpe_norm': 'FACTURE'},
 {'td001_dpe_id': '1009804',
  'gen_ch_lib_ft': 'pac air/air',
  'vr_source_ft': 'installation de chauffage  283 8\xa0m²)\xa0:\r - chauffage 1 : pac air air split m

In [76]:
m_ch.loc[m_ch.gen_ch_lib_ft.fillna('na').str.contains('air/eau')]

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm


In [77]:
m_ch.gen_ch_lib_<Q<

,td001_dpe_id,gen_ch_lib_ft,vr_source_ft,type_installation_ch_ft,energie_ch_ft,gen_ch_lib_desc,vr_source_desc,type_installation_ch_desc,energie_ch_desc,gen_ch_lib_infer_concat,mix_energetique_ch,type_installation_ch_concat,nom_methode_dpe_norm
0,1000015,convecteurs electriques nfc,convecteur electrique nfc :\r - type de produ...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteur electrique nfc\r,indetermine,indetermine,convecteurs electriques nfc,Electricité non renouvelable,Chauffage Individuel,3CL 2012
1,1000040,convecteurs electriques nfc,convecteurs electriques nfc systeme individue...,Chauffage Individuel,Electricité non renouvelable,convecteurs electriques nfc,convecteurs electriques nfc systeme individuel),Chauffage Individuel,Chauffage Individuel,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
2,1000043,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee apres 1991,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
3,1000076,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur<br ...,indetermine,indetermine,panneaux rayonnants electriques nfc,panneaux rayonnants nfc avec programmateur,indetermine,indetermine,autres emetteurs a effet joule,Electricité non renouvelable,Chauffage Individuel,3CL 2012
4,1000112,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Fioul domestique,chaudiere fioul indetermine,chaudiere individuelle fioul installee entre 1...,Chauffage Individuel,Chauffage Individuel,NaN,NaN,NaN,FACTURE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145974,9402027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145975,9410001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE
145976,9421242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3CL 2012
145977,9421904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACTURE


In [61]:
m_ch.gen_ch_lib_ft.value_counts().to_dict()

{'reseau de chaleur': 20592,
 'chaudiere gaz indetermine': 18914,
 'indetermine': 11751,
 'chaudiere gaz basse temperature': 10978,
 'chaudiere gaz standard': 10088,
 'convecteurs electriques nfc': 9708,
 'chaudiere gaz condensation': 8251,
 'autres emetteurs a effet joule': 8158,
 'radiateurs electriques': 6936,
 'panneaux rayonnants electriques nfc': 4599,
 'chaudiere fioul indetermine': 3706,
 'chaudiere energie indetermine indetermine': 2772,
 'chaudiere fioul standard': 1348,
 'pac air/air': 1155,
 'chaudiere energie indetermine basse temperature': 647,
 'chaudiere energie indetermine condensation': 454,
 'chaudiere fioul basse temperature': 441,
 'pac eau/eau': 397,
 'plafonds/planchers rayonnants electriques nfc': 386,
 'convecteurs electriques nfc + panneaux rayonnants electriques nfc': 246,
 'poele ou insert bois': 230,
 'convecteurs electriques nfc + radiateurs electriques': 175,
 'chaudiere bois': 120,
 'pac geothermique': 116,
 'panneaux rayonnants electriques nfc + radiate

In [53]:
s=m_ch.loc[m_ch.nom_methode_dpe_norm=='FACTURE'].gen_ch_lib_ft.fillna('indetermine').value_counts()

In [57]:
s.loc['indetermine']/s.sum()

0.15338467987006182

In [55]:
(s=='indetermine').mean()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


0.0

In [35]:
m_ch.groupby('nom_methode_dpe_norm').gen_ch_lib_ft.value_counts()

nom_methode_dpe_norm  gen_ch_lib_ft                                                              
3CL 2005              convecteurs electriques nfc                                                    45
                      radiateurs electriques                                                         15
                      panneaux rayonnants electriques nfc                                            13
                      chaudiere gaz basse temperature                                                12
                      chaudiere gaz standard                                                          9
                                                                                                     ..
FACTURE               chaudiere energie indetermine indetermine + convecteurs electriques nfc         1
                      chaudiere gaz basse temperature + chaudiere gaz indetermine                     1
                      chaudiere gaz indetermine + indetermine         

In [ ]:
m.type_installation_ch_ft.value_counts()

In [ ]:
m.loc[m.gen_ch_lib_ft=='indetermine'].head(470).to_dict(orient='records')